## vcoabuarly 만들기

In [6]:
data_path = "../data/books_txt_full/Fantasy"

from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords

# 텍스트 파일의 내용을 변수 text로 리턴하는 함수
def load_doc(filename):
    # read only로 파일을 엽니다.
    file = open(filename, 'r', errors='replace')
    # 모든 텍스트를 읽습니다.
    text = file.read()
    # 파일을 닫습니다.
    file.close()
    return text

def clean_doc(doc):
    # white space 기준으로 tokenize 합니다.
    tokens = doc.split()
    # 각 token에서 모든 구두점을 삭제합니다.
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # 각 token에서 alaphabet으로만 이루어지지 않은 모든 단어를 삭제합니다.
    tokens = [word for word in tokens if word.isalpha()]
    # 각 token에서 stopwrods를 삭제합니다.
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # 각 token에서 1글자 이하인 모든 단어를 삭제합니다.
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

# 텍스트 파일을 불러와서 vocab에 추가하는 함수
def add_doc_to_vocab(filename, vocab):
    # 텍스트 파일을 불러옵니다.
    doc = load_doc(filename)
    # 텍스트 파일을 clean toekn으로 리턴합니다.
    tokens = clean_doc(doc)
    # clean token을 vocab에 추가합니다.
    vocab.update(tokens)

# 폴더에 있는 모든 문서를 vocab에 추가하는 함수
def process_docs(directory, vocab, is_train):
    # 폴더에 있는 모든 파일을 순회합니다.
    for filename in listdir(directory):
        # 인덱스가 새겨진 파일 이름과 is_train 인자를 기준으로 test set으로 분류할 모든 파일을 건너뜁니다.
        if is_train and filename.startswith('cv9'):
            continue
        if not is_train and not filename.startswith('cv9'):
            continue
        # 폴더에 있는 파일의 절대 경로를 구합니다.
        path = directory + '/' + filename
        # 텍스트 파일을 불러와서 vocab에 추가하는 함수를 실행합니다.
        add_doc_to_vocab(path, vocab)
    
def save_list(lines, filename):
    # 각 문장을 하나의 텍스트 일부로 바꿉니다.
    data = '\n'.join(lines)
    # 파일을 쓰기 모드로 엽니다.
    file = open(filename, 'w')
    # 변환한 텍스트를 파일에 씁니다.
    file.write(data)
    # 파일을 닫습니다.
    file.close()
    
# vocab을 Counter() 객체로 할당합니다.
vocab = Counter()
# 폴더를 지정하고 폴더 내 모든 문서를 vocab에 추가합니다.
process_docs(data_path, vocab, True)
# vocab의 크기를 출력합니다.
print(len(vocab))
# vocab에서 가장 많이 등장한 50개 단어를 출력합니다.
print(vocab.most_common(50))

# token을 min_occurence 기준으로 유지합니다.
min_occurence = 2
tokens = [k for k,c in vocab.items() if c >= min_occurence]
print(len(tokens))
# token을 vocab 파일로 저장합니다.
save_list(tokens, 'fantasy_vocab.txt')

500188
[('He', 673998), ('The', 640934), ('said', 615681), ('She', 463896), ('would', 366837), ('back', 365738), ('one', 358634), ('like', 353855), ('could', 345128), ('It', 289836), ('know', 268834), ('You', 244781), ('eyes', 241675), ('And', 232197), ('But', 222899), ('time', 222713), ('didnt', 219388), ('get', 188022), ('around', 187870), ('way', 185794), ('head', 183696), ('Im', 181330), ('looked', 180578), ('see', 178171), ('going', 166375), ('hand', 165042), ('even', 163081), ('away', 161169), ('us', 158463), ('think', 157051), ('dont', 156430), ('right', 155481), ('face', 149820), ('still', 148025), ('asked', 146455), ('thought', 144511), ('What', 142710), ('want', 142677), ('go', 142576), ('They', 141863), ('something', 138943), ('made', 135128), ('His', 133491), ('never', 133176), ('man', 130641), ('much', 129293), ('little', 126789), ('knew', 124890), ('No', 122911), ('We', 122225)]
221364


## Word2Vec 만들기

In [7]:
from string import punctuation
from os import listdir
from gensim.models import Word2Vec

# 텍스트 파일의 내용을 변수 text로 리턴하는 함수
def load_doc(filename):
    # read only로 파일을 엽니다.
    file = open(filename, 'r', errors='replace')
    # 모든 텍스트를 읽습니다.
    text = file.read()
    # 파일을 닫습니다.
    file.close()
    return text


# 문서를 cleaned lines(=tokens)으로 리턴합니다.
# tokens = line, token = word
def doc_to_clean_lines(doc, vocab):
    clean_lines = []
    lines = doc.splitlines()
    for line in lines:
        # 공백 기준으로 token을 나눕니다.
        tokens = line.split()
        # 구두점 제거
        table = str.maketrans('', '', punctuation)
        tokens = [w.translate(table) for w in tokens]
        # vocab에 있는 단어만 추출합니다.
        tokens = [w for w in tokens if w in vocab]
        clean_lines.append(tokens)
    return clean_lines


# 폴더에 있는 모든 문서를 vocab에 추가하는 함수
def process_docs(directory, vocab, is_train):
    lines = list()
    # 폴더에 있는 모든 파일을 순회합니다.
    for filename in listdir(directory):
        # 인덱스가 새겨진 파일 이름과 is_train 인자를 기준으로 test set으로 분류할 모든 파일을 건너뜁니다.
        if is_train and filename.startswith('cv9'):
            continue
        if not is_train and not filename.startswith('cv9'):
            continue
        # 폴더에 있는 파일의 절대 경로를 구합니다.
        path = directory + '/' + filename
        # 파일을 불러옵니다.
        doc = load_doc(path)
        # 파일을 개끗하게 만들어서 token으로 리턴합니다.
        doc_lines = doc_to_clean_lines(doc, vocab)
        # 리스트 객체에 추가합니다.
        lines += doc_lines
    return lines

# 보카를 불러옵니다.
vocab_filename = 'fantasy_vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

# 모든 training set을 불러옵니다.
train_docs = process_docs(data_path, vocab, True)
sentences = train_docs
print('Total training sentences: %d' % len(sentences))

# word2vec 모델을 훈련시킵니다.
model = Word2Vec(sentences, size=100, window=5, workers=8, min_count=1)
# 모델의 vocabulary size를 요약합니다.
words = list(model.wv.vocab)
print("Vocabulary size: %d" % len(words))

# 모델을 ASCII 포맷으로 저장합니다.
filename = 'fantasy_embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)

Total training sentences: 7553656
Vocabulary size: 221364


## Pre-trained Word2vec 사용하기